# PYTHON - Final Exam 
### Pierre YVENOU - Paul GRAVIS
SKEMA BS - 25-26

## Imports

In [145]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# historical stock data
historical_stock_data_df = pd.read_csv('tech_companies_stock_prices-2.csv')
# historical performance data
historical_performance_pt_mgmt_df = pd.read_csv('test_portfolio.csv')

## Cleaning data  (historical_stock_data_df)

In [146]:
historical_stock_data_df.drop(columns=['Unnamed: 0.1','Unnamed: 0'], inplace=True)
historical_stock_data_df.head(10)
historical_stock_data_df

,Date,Stock_name,Open,Close,Close_1,Open_1,Close_2,Open_2,Close_3,Open_3,Close_4,Open_4,Close_5,Open_5
0,2023-10-09,AAPL,176.809998,178.990005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-10,AAPL,178.100006,178.389999,178.990005,176.809998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-11,AAPL,178.199997,179.800003,178.389999,178.100006,178.990005,176.809998,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-12,AAPL,180.070007,180.710007,179.800003,178.199997,178.389999,178.100006,178.990005,176.809998,NaN,NaN,NaN,NaN
4,2023-10-13,AAPL,181.419998,178.850006,180.710007,180.070007,179.800003,178.199997,178.389999,178.100006,178.990005,176.809998,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815,2024-10-14,NVDA,136.470001,138.070007,134.800003,134.009995,134.809998,131.910004,132.649994,134.110001,132.889999,130.259995,127.720001,124.989998
1816,2024-10-15,NVDA,137.869995,131.600006,138.070007,136.470001,134.800003,134.009995,134.809998,131.910004,132.649994,134.110001,132.889999,130.259995
1817,2024-10-16,NVDA,133.979996,135.720001,131.600006,137.869995,138.070007,136.470001,134.800003,134.009995,134.809998,131.910004,132.649994,134.110001
1818,2024-10-17,NVDA,139.339996,136.929993,135.720001,133.979996,131.600006,137.869995,138.070007,136.470001,134.800003,134.009995,134.809998,131.910004


## Cleaning data  (historical_performance_pt_mgmt_df)

In [147]:
historical_performance_pt_mgmt_df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
historical_performance_pt_mgmt_df.head(10)

,Date,TSLA,AAPL,AMZN,MSFT,NVDA,META,GOOGL,cash,value
0,2023-10-09,0.0,561.0,201.0,333.0,0.0,0.0,489.0,333.959930,304045.053452
1,2023-10-10,0.0,4.0,-2.0,-2.0,0.0,0.0,1.0,68.659973,303305.982361
2,2023-10-11,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,129.740005,307122.227936
3,2023-10-12,0.0,-1.0,-2.0,4.0,0.0,0.0,-5.0,-172.620010,306555.915863
4,2023-10-13,0.0,-3.0,-1.0,-1.0,0.0,0.0,7.0,33.959961,303080.545944
5,2023-10-16,0.0,7.0,1.0,-3.0,0.0,0.0,-2.0,-98.450043,306050.858948
6,2023-10-17,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,-52.329987,305059.516388
7,2023-10-18,0.0,4.0,1.0,-2.0,0.0,0.0,-2.0,111.659973,302161.527100
8,2023-10-19,0.0,-1.0,-1.0,0.0,0.0,0.0,3.0,-108.889999,302295.104858
9,2023-10-20,0.0,-1.0,3.0,-2.0,0.0,0.0,2.0,179.939987,297603.429611


## debugging purposes

In [148]:
print("Stock Data Columns:", historical_stock_data_df.columns)
print("Performance Data Columns:", historical_performance_pt_mgmt_df.columns)

Stock Data Columns: Index(['Date', 'Stock_name', 'Open', 'Close', 'Close_1', 'Open_1', 'Close_2',
       'Open_2', 'Close_3', 'Open_3', 'Close_4', 'Open_4', 'Close_5',
       'Open_5'],
      dtype='object')
Performance Data Columns: Index(['Date', 'TSLA', 'AAPL', 'AMZN', 'MSFT', 'NVDA', 'META', 'GOOGL', 'cash',
       'value'],
      dtype='object')


## Step 1: Reconstruct the portfolio (at the end of each day)
- Numbers of stocks and cash

In [149]:
stock_names = historical_performance_pt_mgmt_df.columns.tolist()
stock_names
exclude = ['Date', 'cash', 'value']
stock_names = [s for s in stock_names if s not in exclude]
stock_names

['TSLA', 'AAPL', 'AMZN', 'MSFT', 'NVDA', 'META', 'GOOGL']

In [150]:
merged_df = pd.merge(historical_stock_data_df, historical_performance_pt_mgmt_df, on='Date', how='inner')
for stock in stock_names:
    merged_df[stock + '_Value'] = merged_df[stock] * merged_df['Close']
    merged_df[stock + '_Amount'] = merged_df[stock].cumsum()
merged_df.head(10)

,Date,Stock_name,Open,Close,Close_1,Open_1,Close_2,Open_2,Close_3,Open_3,...,AMZN_Value,AMZN_Amount,MSFT_Value,MSFT_Amount,NVDA_Value,NVDA_Amount,META_Value,META_Amount,GOOGL_Value,GOOGL_Amount
0,2023-10-09,AAPL,176.809998,178.990005,NaN,NaN,NaN,NaN,NaN,NaN,...,35976.991104,201.0,59603.671829,333.0,0.0,0.0,0.0,0.0,87526.112686,489.0
1,2023-10-10,AAPL,178.100006,178.389999,178.990005,176.809998,NaN,NaN,NaN,NaN,...,-356.779999,199.0,-356.779999,331.0,0.0,0.0,0.0,0.0,178.389999,490.0
2,2023-10-11,AAPL,178.199997,179.800003,178.389999,178.100006,178.990005,176.809998,NaN,NaN,...,-179.800003,198.0,0.000000,331.0,0.0,0.0,0.0,0.0,0.000000,490.0
3,2023-10-12,AAPL,180.070007,180.710007,179.800003,178.199997,178.389999,178.100006,178.990005,176.809998,...,-361.420013,196.0,722.840027,335.0,0.0,0.0,0.0,0.0,-903.550034,485.0
4,2023-10-13,AAPL,181.419998,178.850006,180.710007,180.070007,179.800003,178.199997,178.389999,178.100006,...,-178.850006,195.0,-178.850006,334.0,0.0,0.0,0.0,0.0,1251.950043,492.0
5,2023-10-16,AAPL,176.750000,178.720001,178.850006,181.419998,180.710007,180.070007,179.800003,178.199997,...,178.720001,196.0,-536.160004,331.0,0.0,0.0,0.0,0.0,-357.440002,490.0
6,2023-10-17,AAPL,176.649994,177.149994,178.720001,176.750000,178.850006,181.419998,180.710007,180.070007,...,0.000000,196.0,177.149994,332.0,0.0,0.0,0.0,0.0,-354.299988,488.0
7,2023-10-18,AAPL,175.580002,175.839996,177.149994,176.649994,178.720001,176.750000,178.850006,181.419998,...,175.839996,197.0,-351.679993,330.0,0.0,0.0,0.0,0.0,-351.679993,486.0
8,2023-10-19,AAPL,176.039993,175.460007,175.839996,175.580002,177.149994,176.649994,178.720001,176.750000,...,-175.460007,196.0,0.000000,330.0,0.0,0.0,0.0,0.0,526.380020,489.0
9,2023-10-20,AAPL,175.309998,172.880005,175.460007,176.039993,175.839996,175.580002,177.149994,176.649994,...,518.640015,199.0,-345.760010,328.0,0.0,0.0,0.0,0.0,345.760010,491.0


In [151]:
# we try with one example (because hard to see if 
# it's working above with all the columns/stocks at the same time)
test_aapl_df = merged_df[['Date','AAPL', 'Close', 'AAPL_Value', 'AAPL_Amount']]
test_aapl_df.head(10)

,Date,AAPL,Close,AAPL_Value,AAPL_Amount
0,2023-10-09,561.0,178.990005,100413.393082,561.0
1,2023-10-10,4.0,178.389999,713.559998,565.0
2,2023-10-11,0.0,179.800003,0.000000,565.0
3,2023-10-12,-1.0,180.710007,-180.710007,564.0
4,2023-10-13,-3.0,178.850006,-536.550018,561.0
5,2023-10-16,7.0,178.720001,1251.040009,568.0
6,2023-10-17,0.0,177.149994,0.000000,568.0
7,2023-10-18,4.0,175.839996,703.359985,572.0
8,2023-10-19,-1.0,175.460007,-175.460007,571.0
9,2023-10-20,-1.0,172.880005,-172.880005,570.0


## Step 2: Use models to perform predictions 
Models: 
- Linear regression
- SVM
- Decision trees
- Neural networks

In [152]:
# pivot to wide format for market data
market_pivot = historical_stock_data_df.pivot(index='Date', columns='Stock_name', values='Open')
market_pivot.columns = [f"{col}_Open" for col in market_pivot.columns]
market_pivot.index = pd.to_datetime(market_pivot.index)


# Reconstruct t-1 state & ensure date is datetime
historical_performance_pt_mgmt_df['Date'] = pd.to_datetime(historical_performance_pt_mgmt_df['Date'])
historical_performance_pt_mgmt_df.set_index('Date', inplace=True)

# filter (same as above) to have only stock columns
stock_cols = [c for c in historical_performance_pt_mgmt_df.columns if c not in ['cash', 'value']]
current_positions = historical_performance_pt_mgmt_df[stock_cols].cumsum()
current_cash = historical_performance_pt_mgmt_df['cash'].cumsum()

# Shift by 1 to get "Previous Day's" state (Feature Set 1)
X_positions = current_positions.shift(1).add_suffix('_pos') 
X_cash = current_cash.shift(1).to_frame(name='Cash_prev')
X_market = market_pivot.reindex(X_positions.index)
X = pd.concat([X_positions, X_cash, X_market], axis=1)

y = historical_performance_pt_mgmt_df['value']

# Drop NaNs 
final_df = pd.concat([X, y], axis=1).dropna()

X_final = final_df.drop(columns=['value'])
y_final = final_df['value']

print("Shape of X:", X_final.shape)
print("First few rows of X:")
print(X_final.head())

Shape of X: (259, 15)
First few rows of X:
            TSLA_pos  AAPL_pos  AMZN_pos  MSFT_pos  NVDA_pos  META_pos  \
Date                                                                     
2023-10-10       0.0     561.0     201.0     333.0       0.0       0.0   
2023-10-11       0.0     565.0     199.0     331.0       0.0       0.0   
2023-10-12       0.0     565.0     198.0     331.0       0.0       0.0   
2023-10-13       0.0     564.0     196.0     335.0       0.0       0.0   
2023-10-16       0.0     561.0     195.0     334.0       0.0       0.0   

            GOOGL_pos   Cash_prev   AAPL_Open   AMZN_Open  GOOGL_Open  \
Date                                                                    
2023-10-10      489.0  333.959930  178.100006  128.820007  138.500000   
2023-10-11      490.0  402.619904  178.199997  129.740005  138.580002   
2023-10-12      490.0  532.359909  180.070007  132.169998  141.050003   
2023-10-13      485.0  359.739899  181.419998  132.979996  139.380005   


In [153]:
# Splitting data for test purposes
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, shuffle=False)

# Scale data (Important for SVM and Neural Networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    "Linear Regression": LinearRegression(),
    "SVM": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

print("\n--- Model Performance (Portfolio Value) ---")
for name, model in models.items():
    if name in ["SVM", "Neural Network"]:
        model.fit(X_train_scaled, y_train)
        preds = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
    
    score = r2_score(y_test, preds)
    print(f"{name}: R2 Score = {score:.4f}")


--- Model Performance (Portfolio Value) ---
Linear Regression: R2 Score = 0.7836
SVM: R2 Score = -31.4286
Decision Tree: R2 Score = -7.4551
Neural Network: R2 Score = -2206.5548


/Users/pierre/Desktop/COURS SKEMA/Python/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [154]:
# --- 1. DATA PREPARATION ---

# recreate safely the actions dataframe
action_cols = [c for c in historical_performance_pt_mgmt_df.columns if c not in ['cash', 'value']]
actions_df = historical_performance_pt_mgmt_df[action_cols]

# We need to predict today's action (t) using t-1, t-2, t-3
features = []
targets = actions_df.copy()

for lag in [1, 2, 3]:
    # Shift data by 'lag' days and rename columns
    lagged_data = actions_df.shift(lag)
    lagged_data.columns = [f"{col}_lag{lag}" for col in action_cols]
    features.append(lagged_data)

X = pd.concat(features, axis=1)
y = targets

# Drop NaNs resulting from lagging
data = pd.concat([X, y], axis=1).dropna()

X_final = data.iloc[:, :X.shape[1]]
y_final = data.iloc[:, X.shape[1]:]

print(f"Input Shape: {X_final.shape} (Lags)")
print(f"Target Shape: {y_final.shape} (Actions)")

Input Shape: (257, 21) (Lags)
Target Shape: (257, 7) (Actions)


In [155]:
# --- 2. MODELING (Part B) ---

# Split Data (Keep 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, shuffle=False)

# Scale data (Actions can range from -100 to +100, scaling helps SVM/NN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Models
# Note: SVR does not support multi-output natively, so we wrap it in MultiOutputRegressor.
# Decision Trees and Neural Networks usually handle it automatically, but the wrapper is safer.
models = {
    "Linear Regression": LinearRegression(),
    "SVM": MultiOutputRegressor(SVR()), 
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

print("\n--- Model Performance (Trading Actions) ---")
for name, model in models.items():
    # Train
    if name in ["SVM", "Neural Network"]:
        model.fit(X_train_scaled, y_train)
        score = model.score(X_test_scaled, y_test)
    else:
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
    
    # Score is the coefficient of determination R^2 of the prediction
    print(f"{name}: R2 Score = {score:.4f}")


--- Model Performance (Trading Actions) ---
Linear Regression: R2 Score = 0.3264
SVM: R2 Score = 0.3829
Decision Tree: R2 Score = -0.2081
Neural Network: R2 Score = -0.1716


# Conclusion
### Comparison of models efficiency (for step 1)

- The Linear Regression model significantly outperformed all others with an R^2 of 0.7836. 
- We can assume this is because the portfolio value is a linear combination of asset positions and prices. 
- In the other hand, the SVM, Decision Tree, and Neural Network models failed (R2<0) because they struggled to extrapolate beyond the specific price ranges seen in the training data, leading to poor generalization on the test set.

### Comparison of models efficiency (for step 2)
- The SVM model proved to be the most effective for predicting trading actions, achieving an R^2 score of 0.3829, closely followed by Linear Regression (0.33). 
- This indicates that the portfolio management algorithm follows a recognizable, though not perfectly deterministic, pattern based on its recent history.
- Conversely, the Decision Tree and Neural Network models produced negative R^2 scores, indicating severe overfitting. 
- These complex models memorized noise in the training data rather than learning the underlying trading strategy. 
- Therefore, SVM is selected as the best model for predicting trading behavior.